# 06 - Test and Deploy Training Pipeline to Vertex AI Pipelines

The purpose of this notebook is to compile and run the TFX pipeline to AI Platform Managed Pipelines. The notebook covers the following tasks:
1. Test the pipeline locally using local runner.
2. Set the pipeline deployment configuration.
3. Build Container Image
4. Compile TFX Pipeline
5. Submit a pipeline job to AI Platform Pipelines (Managed).

## Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import kfp
import tfx

print("Tensorflow Version:", tfx.__version__)
print("KFP Version:", kfp.__version__)

In [ ]:
!rm -r src/raw_schema/.ipynb_checkpoints/

In [ ]:
PROJECT = 'ksalama-cloudml' # Change to your project Id.
REGION = 'us-central1'
BUCKET = 'ksalama-cloudml-us' # Change to your bucket.

CICD_IMAGE_NAME = 'cicd:latest'
CICD_IMAGE_URI = f"gcr.io/{PROJECT}/{CICD_IMAGE_NAME}"

## Build CI/CD  Container Image for Cloud Build

This is the runtime environment where the steps of testing and deploying the model will be executed.

In [ ]:
!echo $CICD_IMAGE_URI

In [ ]:
!gcloud builds submit --tag $CICD_IMAGE_URI build/. --timeout=15m

## 1. Run the Pipeline CICD steps locally

### Set pipeline configurations for the local run

In [ ]:
os.environ["DATASET_DISPLAY_NAME"] = 'chicago_taxi_tips'
os.environ["MODEL_DISPLAY_NAME"]  =  'chicago_taxi_tips_classifier_v1'
os.environ["PROJECT"] = 'ksalama-cloudml'
os.environ["REGION"] = 'us-central1'
os.environ["GCS_LOCATION"] = f"gs://{BUCKET}/ucaip_demo/chicago_taxi/pipelines_e2e_test"
os.environ["TRAIN_LIMIT"] = "1000"
os.environ["TEST_LIMIT"] = "100"
os.environ["UPLOAD_MODEL"] = "FALSE"
os.environ["ACCURACY_THRESHOLD"] = "0.1"
os.environ["BEAM_RUNNER"] = "DirectRunner"
os.environ["TRAINING_RUNNER"] = "local"

In [ ]:
from src.pipelines import config
for key, value in config.__dict__.items():
    if key.isupper(): print(f'{key}: {value}')

### Run e2e pipeline test

In [ ]:
!py.test src/tests/pipeline_deployment_tests.py::test_e2e_pipeline -s

### Set the pipeline configurations for the Vertex AI run

In [ ]:
VERSION = 'tfx-0-30'
os.environ["DATASET_DISPLAY_NAME"] = 'chicago_taxi_tips'
os.environ["PIPELINE_NAME"] = f'chicago_taxi_tips_train_pipeline_{VERSION}'
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION
os.environ["GCS_LOCATION"] = f"gs://{BUCKET}/ucaip_demo/chicago_taxi/pipelines_managed_runner"
os.environ["TRAIN_LIMIT"] = "85000"
os.environ["TEST_LIMIT"] = "15000"
os.environ["BEAM_RUNNER"] = "DataflowRunner"
os.environ["TRAINING_RUNNER"] = "caip"
os.environ["IMAGE_URI"] = f"gcr.io/{PROJECT}/chicago_taxi_tips:{VERSION}"

In [ ]:
from src.pipelines import config

import importlib
importlib.reload(config)

for key, value in config.__dict__.items():
    if key.isupper(): print(f'{key}: {value}')

### Build container image

This is the tfx runtime environment for the training pipeline steps.

In [ ]:
!echo $IMAGE_URI

In [ ]:
!gcloud builds submit --tag $IMAGE_URI . --timeout=15m

### Compile pipeline

In [ ]:
from src.pipelines import runner

pipeline_definition_file = runner.compile_pipeline()

### Submit run to AI Platform Managed Pipelines

In [ ]:
from kfp.v2.google.client import AIPlatformClient

pipeline_client = AIPlatformClient(
    project_id=PROJECT, region=REGION)
                 
pipeline_client.create_run_from_job_spec(
    job_spec_path=pipeline_definition_file,
    parameter_values={
        'learning_rate': 0.003,
        'batch_size': 512,
        'hidden_units': '128,128',
        'num_epochs': 30,
    }
)



![Pipeline execution](imgs/managed-pipeline.png)

## 2. Execute the Model Deployment CI/CD rountine in Cloud Build